# Introduction

## Description

Btgym is an OpenAI Gym-compatible environment for Backtrader backtesting/trading library, designed to provide gym-integrated framework for running reinforcement learning experiments in real world algorithmic trading environments. It is based on two libraries:
* Backtrader is open-source algorithmic trading library.
* OpenAI Gym is…, well, everyone knows Gym:

## Problem definition

* **Discrete actions setup**: consider setup with one riskless asset acting as broker account cash and K (by default - one) risky assets. For every risky asset there exists track of historic price records referred as *data-line*. Apart from assets data lines there optionally exists number of exogenous data lines holding some information and statistics, e.g. economic indexes, encoded news, macroeconomic indicators, weather forecasts, ... which are considered relevant to decision-making. It is supposed for this setup that:
    * there is no interest rates for any asset;
    * broker actions are fixed-size market orders (buy, sell, close); 
    * short selling is permitted;
    * transaction costs are modelled via broker commission;
    * ‘market liquidity’ and ‘capital impact’ assumptions are met; 
    * time indexes match for all data lines provided;
* **The problem is modelled as discrete-time finite-horizon partially observable Markov decision process for equity/currency trading**:
    * for every asset traded agent action space is discrete (0: hold [do nothing], 1:buy, 2: sell, 3:close [position]);
    * environment is episodic: maximum episode duration and episode termination conditions are set;
    * for every timestep of the episode agent is given environment state observation as tensor of last m time-embedded preprocessed values for every data-line included and emits actions according some stochastic policy.
    * agent’s goal is to maximize expected cumulative capital by learning optimal policy;

## Environment engine description

BTgym uses Backtrader framework for actual environment computations. In short:
* User defines backtrading engine parameters by composing Backtrader.Cerebro() subclass, provides historic prices dataset as BTgymDataset() instance and passes it as arguments when making BTgym environment. See https://www.backtrader.com/docu/concepts.html for general Backtrader concepts descriptions.
* Environment starts separate server process responsible for rendering gym environment queries like env.reset() and env.step() by repeatedly sampling episodes form given dataset and running backtesting Cerebro engine on it. See OpenAI Gym documentation for details: https://gym.openai.com/docs

Workflow sample:
* Define backtesting BTgymStrategy(bt.Strategy), which will control Environment inner dynamics and backtesting logic.
    * For RL-specific part, any STATE, REWARD, DONE and INFO computation logic can be implemented by overriding get_state(), get_reward(), get_info(), is_done() and set_datalines() methods.
    * For Broker/Trading specific part, custom order execution logic, stake sizing, analytics tracking can be implemented as for regular bt.Strategy().
* Instantiate Cerbro(), add BTgymStrategy(), backtrader Sizers, Analyzers and Observers (if needed).
* Define dataset by passing CSV datafile and parameters to BTgymDataset instance.
    * BTgymDataset() is simply Backtrader.feeds class wrapper, which pipes CSV [source]->pandas[for efficient sampling]-> bt.feeds routine and implements random episode data sampling.
* Initialize (or register and make()) gym environment with Cerebro() and BTgymDataset() along with other kwargs.
* Run your favorite RL algorithm:
    * start episode by calling env.reset();
    * advance one step of episode by calling env.step(), perform agent training or testing;
    * after single episode is finished, retrieve agent performance statistic by env.get_stat().

## Server operation details

Backtrader server starts when env.reset() method is called for first time, runs as separate process, follows simple Request/Reply pattern (every request should be paired with reply message) and operates one of two modes:
* Control mode: initial mode, accepts only _reset, _stop and _getstat messages. Any other message is ignored and replied with simple info messge. Shuts down upon recieving _stop via env._stop_server() method, goes to episode mode upon _reset (via env.reset()) and send last run episode statistic (if any) upon _getstat via env.get_stat().
* Episode mode: runs episode according BtGymStrategy() logic. Accepts action messages, returns tuple: ([state observation], [reward], [is_done], [aux.info]). Finishes episode upon recieving action`==`_done or according to strategy termination rules, than falls back to control mode.

Before every episode start, BTserver samples episode data and adds it to bt.Cerebro() instance
along with specific _BTgymAnalyzer. The service of this hidden Analyzer is twofold:
enables strategy-environment communication by calling RL-related BTgymStrategy methods:
get_state(), get_reward(), get_info() and is_done() [see below];
controls episode termination conditions.
Episode runtime: after preparing environment initial state by running BTgymStrategy start(), prenext() methods, server halts and waits for incoming agent action. Upon receiving action, server performs all necessary next() computations (e.g. issues orders, computes broker values etc.), composes environment response and sends it back to agent ( via _BTgymAnalyzer). Actually, since ‘no market impact’ is assumed, all state computations are performed one step ahead:

## Links

* https://github.com/Kismuz/btgym
* http://github.com/mementum/backtrader
* http://github.com/openai/gym
* https://kismuz.github.io/btgym/
* https://www.backtrader.com/docu/index.html

# Quickstart

Making gym environment with all parmeters set to defaults is as simple as:

In [1]:
from btgym import BTgymEnv

MyEnvironment = BTgymEnv(filename='btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv')
MyEnvironment.close()

ImportError: cannot import name 'BTgymEnv' from 'btgym' (unknown location)

Adding more controls may look like:

In [2]:
from gym import spaces
from btgym import BTgymEnv

MyEnvironment = BTgymEnv(filename='btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
                         episode_duration={'days': 2, 'hours': 23, 'minutes': 55},
                         drawdown_call=50,
                         state_shape={'raw': spaces.Box(low=0,high=1,shape=(20,4))},
                         port=5555,
                         verbose=1,
                         )
MyEnvironment.close()

BTgymDataset class is DEPRECATED, use btgym.datafeed.derivative.BTgymDataset2 instead.
[2020-06-05 23:21:13.019786] INFO: BTgymAPIshell_0: Base Dataset class used.
[2020-06-05 23:21:13.020492] INFO: BTgymAPIshell_0: Connecting data_server...
[2020-06-05 23:21:13.086284] INFO: BTgymDataServer_0: PID: 5595
[2020-06-05 23:21:13.457120] INFO: SimpleDataSet_0: Loaded 372678 records from <btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2020-06-05 23:21:13.551928] INFO: SimpleDataSet_0: Data summary:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092040       1.092140       0.0
50%         1.113530       1.113610      

Same one but registering environment in Gym preferred way:

In [5]:
import gym
from gym import spaces
from btgym import BTgymEnv

env_params = dict(filename='btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
                  episode_duration={'days': 2, 'hours': 23, 'minutes': 55},
                  drawdown_call=50,
                  state_shape={'raw': spaces.Box(low=0,high=1,shape=(20,4))},
                  port=5555,
                  verbose=1,
                  )

gym.envs.register(id='backtrader-v52555', entry_point='btgym:BTgymEnv', kwargs=env_params,)

MyEnvironment = gym.make('backtrader-v52555')
MyEnvironment.close()

BTgymDataset class is DEPRECATED, use btgym.datafeed.derivative.BTgymDataset2 instead.
[2020-06-05 23:23:04.810738] INFO: BTgymAPIshell_0: Base Dataset class used.
[2020-06-05 23:23:04.811689] INFO: BTgymAPIshell_0: Connecting data_server...
[2020-06-05 23:23:04.924349] INFO: BTgymDataServer_0: PID: 5678
[2020-06-05 23:23:05.289550] INFO: SimpleDataSet_0: Loaded 372678 records from <btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2020-06-05 23:23:05.355703] INFO: SimpleDataSet_0: Data summary:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092040       1.092140       0.0
50%         1.113530       1.113610      

Maximum environment flexibility is achieved by explicitly defining and passing Dataset and Cerebro instances:

In [8]:
from gym import spaces
import backtrader as bt
from btgym import BTgymDataset, BTgymBaseStrategy, BTgymEnv

MyCerebro = bt.Cerebro()
MyCerebro.addstrategy(BTgymBaseStrategy,
                      state_shape={'raw': spaces.Box(low=0,high=1,shape=(20,4))},
                      skip_frame=5,
                      state_low=None,
                      state_high=None,
                      drawdown_call=50,
                      )

MyCerebro.broker.setcash(100.0)
MyCerebro.broker.setcommission(commission=0.001)
MyCerebro.addsizer(bt.sizers.SizerFix, stake=10)
MyCerebro.addanalyzer(bt.analyzers.DrawDown)

MyDataset = BTgymDataset(filename='btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv',
                         start_weekdays=[0, 1, 2, 4],
                         start_00=True,
                         episode_duration={'days': 0, 'hours': 23, 'minutes': 55},
                         time_gap={'hours': 5},
                         )

MyEnvironment = BTgymEnv(dataset=MyDataset,
                         engine=MyCerebro,
                         port=5555,
                         verbose=1,
                         )
MyEnvironment.close()

BTgymDataset class is DEPRECATED, use btgym.datafeed.derivative.BTgymDataset2 instead.
[2020-06-05 23:23:39.076074] INFO: BTgymAPIshell_0: Custom Dataset class used.
[2020-06-05 23:23:39.076994] INFO: BTgymAPIshell_0: Connecting data_server...
[2020-06-05 23:23:39.158909] INFO: BTgymDataServer_0: PID: 5714
[2020-06-05 23:23:39.531384] INFO: SimpleDataSet_0: Loaded 372678 records from <btgym/examples/data/DAT_ASCII_EURUSD_M1_2016.csv>.
[2020-06-05 23:23:39.600168] INFO: SimpleDataSet_0: Data summary:
                open           high            low          close    volume
count  372678.000000  372678.000000  372678.000000  372678.000000  372678.0
mean        1.107109       1.107198       1.107019       1.107108       0.0
std         0.024843       0.024840       0.024847       0.024844       0.0
min         1.035250       1.035470       1.035220       1.035220       0.0
25%         1.092140       1.092230       1.092040       1.092140       0.0
50%         1.113530       1.113610    

# Development

In [ ]:
# Imports
from btgym import BTgymDataset

## BTgymDataset

In [ ]:
# Load sample data
MyDataset = BTgymDataset(filename='../data/AAPL_2000_2019.csv',
                         start_weekdays=[0, 1, 2, 4],
                         start_00=True,
                         episode_duration={'days': 0, 'hours': 23, 'minutes': 55},
                         time_gap={'hours': 5},
                         )